### Short list of objectives

* Import pdf of Proposed Rule XXXXXXX
* Query rule for baseline responses 
* Fine-tune embedding model, recheck responses
* Summarize rule

In [ ]:
# Set up ollama
!apt-get update && apt-get install tmux vim -y
!pip3 install llama-index llama-parse llama_deploy llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-llms-ollama llama-index-embeddings-ollama llama-index-vector-stores-neo4jvector llama-index-graph-stores-neo4j llama-index-finetuning llama-index-utils-workflow 
!pip3 install sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv graspologic
!pip3 install flash-attn --no-build-isolation
!curl -fsSL https://ollama.com/install.sh | sh


In [ ]:
# Set up Neo4j
!apt install dialog apt-utils -y 
!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | gpg --dearmor -o /etc/apt/keyrings/neotechnology.gpg
!echo 'deb [signed-by=/etc/apt/keyrings/neotechnology.gpg] https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list
!apt list -a neo4j
!add-apt-repository universe -y
!apt install neo4j=1:5.23.0 -y
!echo "neo4j-enterprise neo4j/question select I ACCEPT" | debconf-set-selections
!echo "neo4j-enterprise neo4j/license note" | debconf-set-selections
!apt install openjdk-17-jre -y
!cd /var/lib/neo4j/plugins/ && wget https://github.com/neo4j/apoc/releases/download/5.23.0/apoc-5.23.0-core.jar

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import subprocess, os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

# model_name, ctx_len = "gpt-4o", 128000
# model_name, ctx_len = "llama3.1:latest", 128000
model_name, ctx_len = "hermes3:8b", 128000
# model_name, ctx_len = "gemma2:27b", 8192


if model_name == "gpt-4o":
    openai_key = os.getenv("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = openai_key
    
    print(f"Using OpenAI {model_name}...")
    llm = LOpenAI(model=model_name, max_tokens=4000)
else:
    try: 
        print("Pulling Ollama model...")
        sub_out = subprocess.run(['ollama', 'pull', model_name], capture_output=True, text=True)
    except Exception as e: 
        print(f"Error pulling model: Is the Ollama server running?\n{e}")
    
    addtion_kwargs = {"max_new_tokens": 2000}
    # system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
    llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=False, 
                 request_timeout=4000.0, additional_kwargs=addtion_kwargs) #, system_prompt=system_prompt)
    print(llm.metadata)

Settings.llm = llm

## Import data

In [ ]:
# SEC rules and regulations
!cd /workspace/data && curl -X GET "https://www.ecfr.gov/api/versioner/v1/full/2024-07-23/title-17.xml?chapter=II" -H "accept: application/xml" > title-17.xml

In [3]:
import xml.etree.ElementTree as ET
from sec_utils import get_tree_data

# Path to your XML file
xml_file_path = '/workspace/data/title-17.xml'

# Parse the XML file
tree = ET.parse(xml_file_path)

# Get the root element of the XML document
root = tree.getroot()

sec_data = get_tree_data(root)


In [4]:
from sec_utils import get_metadata
from llama_index.core import Document

documents = [Document(text=t, 
                          text_template='{metadata_str}\n\n{content}',
                          metadata=get_metadata(m, t, metadata={"section":None, "description":None, "mentioned_sections":None})) \
                            for m,t in sec_data.items()]

In [ ]:
from llamaindex_data_utils import extract_text_from_pdf
import os
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

pdf_urls = ["/workspace/data/compliance/FSI_Factsheet.pdf", "/workspace/data/compliance/FSI_Press_Release.pdf", "/workspace/data/compliance/FSI_Rule_proposal.pdf"]
descriptions = ["Factsheet about newly proposed SEC rule.", "Press release regarding newly proposed SEC rule.", "The full text of the newly proposed SEC rule."]
documents_proposal = extract_text_from_pdf(pdf_urls, llama_api_key, llamaparse_kwargs={"split_by_page":False}, save_json_path=None)
# add metadata to the documents_proposal
for i in range(len(documents_proposal)):
    documents_proposal[i].metadata["source"] = pdf_urls[i]
    documents_proposal[i].metadata["description"] = descriptions[i]
    
# documents_proposal.extend(docs)

### Create Distillation Datasets

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=300, chunk_overlap=50)
nodes = parser.get_nodes_from_documents(documents_proposal, show_progress=True)
len(nodes)

In [ ]:
system_prompt = 'Context information is below.\n\n---------------------\n{context_str}\n---------------------\n\nGiven the context information and no prior knowledge, generate {num_questions_per_chunk} questions based on the below query.\n\nYou are a FINRA certified Compliance Specialist that writes exams for firm compliance professionals. Your task is to write precise questions for an upcoming Compliance Officer certification examination. The questions should be diverse across the context with no multiple choice. Restrict the questions to the context information provided."\n'

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.llms.openai import OpenAI
import random

output_path = "/workspace/data/train_dataset_proposal.json"
if not os.path.exists(output_path):
    rand_index = random.sample(range(len(nodes)), len(nodes))
    train_perc=1.0
    train_size = int(len(rand_index)*train_perc)

    train_dataset = generate_qa_embedding_pairs(
        qa_generate_prompt_tmpl = system_prompt,
        num_questions_per_chunk=5,
        save_every=20,
        output_path=output_path,
        llm=llm, 
        nodes=[nodes[x] for x in rand_index[:train_size]],
        verbose=False
    )
    train_dataset.save_json(output_path)

    # val_dataset = generate_qa_embedding_pairs(
    #     qa_generate_prompt_tmpl = system_prompt,
    #     num_questions_per_chunk=5,
    #     save_every=500,
    #     output_path="/workspace/data/val_dataset.json",
    #     llm=llm, 
    #     nodes=[nodes[x] for x in rand_index[train_size:]],
    #     verbose=False
    # )
    # # assert len(val_dataset.relevant_docs) == len(val_dataset.queries)
    # assert (np.unique(list(val_dataset.relevant_docs.values()), return_counts=True)[1]==5).all()
    # val_dataset.save_json("/workspace/data/val_dataset.json")



### Pretrain Embeddings

In [ ]:
%load_ext autoreload
%autoreload 2

#####################
# in llama_index/embeddings/huggingface/base need to add "show_progress_bar=False" to 204, 213
#####################

import os, json

from llama_index.finetuning import EmbeddingAdapterFinetuneEngine, SentenceTransformersFinetuneEngine
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings


train_path = "/workspace/data/train_dataset_proposal.json"
val_path = None #"/workspace/data/val_dataset.json"
if os.path.exists(train_path):
    train_dataset = EmbeddingQAFinetuneDataset.from_json(train_path)
if val_path is not None and os.path.exists(val_path):
    val_dataset = EmbeddingQAFinetuneDataset.from_json(val_path)
else:
    val_dataset = None

embed_model_name = "dunzhang/stella_en_1.5B_v5" #7b params
print("loading embed model...")

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    embed_model_name,
    batch_size=6,
    model_output_path="/workspace/data/rule_proposal_embedding_model",
    val_dataset=val_dataset,
    epochs=4,
    show_progress_bar=True,
    # can optionally pass along any parameters that go into `train_model`
    # optimizer_class=torch.optim.SGD,
    # optimizer_params={"lr": 0.0001, "weight_decay": 0.01}
)


# finetune_engine = EmbeddingAdapterFinetuneEngine(
#     train_dataset,
#     base_embed_model,
#     batch_size=10,
#     model_output_path="/workspace/data/adapter_model",
#     # val_dataset=val_dataset,
#     epochs=4,
#     verbose=False,
#     # can optionally pass along any parameters that go into `train_model`
#     # optimizer_class=torch.optim.SGD,
#     # optimizer_params={"lr": 0.001}
# )

In [ ]:
finetune_engine.finetune(**{"lr": 0.001, "weight_decay": 0.0001})

In [6]:
import gc, torch

# Clear memory
del finetune_engine, train_dataset
gc.collect()
torch.cuda.empty_cache()

## Embedding Model

In [ ]:

import os
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model_name = "dunzhang/stella_en_1.5B_v5" #7b params
finetuned_embed_model_name = "/workspace/data/compliance/rule_proposal_embedding_model_gpt"
print("loading embed model...")
# proposal_embed_model = HuggingFaceEmbedding(model_name=finetuned_embed_model_name)
rules_embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

# Settings.embed_model = embed_model
# Settings.chunk_size = 300
# Settings.chunk_overlap = 50

In [6]:
from typing import Literal

entities = Literal[#"PROPOSED_RULE", 
                   "ACTS",
                   "SECTIONS",
                   "RULES"
                   #"AMENDMENTS"
]

relations = Literal[
    "MENTIONS",
    #"CHANGES",
    # "AMENDS", 
    "REFERS_TO"
]

validation_schema = {
    # "Proposed_Rules": ["CHANGES", "AMENDS"],
    "Acts": ["MENTIONS","REFERS_TO"],
    "Sections": ["MENTIONS","REFERS_TO"],
    "Rules": ["MENTIONS","REFERS_TO"],
    # "Amendments": ["REFERS_TO", "AMENDS"],
}


In [ ]:
from rag_utils import set_neo4j_password, add_lines_to_conf
set_neo4j_password('bewaretheneo')
# add_lines_to_conf()

###### START NEO4J SERVER ######


In [ ]:
# GraphRAG Database
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor, SimpleLLMPathExtractor
from rag_utils import create_neo4j_graph_store, create_neo4j_graphrag, neo4j_query, dump_neo4j_database
import random
from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

graph_idx_persist_dir = "/workspace/data/compliance/graph_idx_testfull"
graph_store_persist_dir= None #"/workspace/data/graph_store"

Settings.chunk_size = 500
Settings.chunk_overlap = 20

# kg_extractor = SchemaLLMPathExtractor(
#     llm=llm,
#     possible_entities=entities,
#     possible_relations=relations,
#     kg_validation_schema=validation_schema,
#     strict=False,  # if false, will allow triples outside of the schema``
#     num_workers=10,
#     max_triplets_per_chunk=10,
# )


# extract_prompt = "You are an expert compliance officer with vast knowledge of SEC Title 17 Chapter II. Your job is to read each section and link mentions of other sections, rules, or acts (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) mentioned. If there are no mentions of other sections, rules, or acts, return an empty list."
# extract_prompt = "You are an expert compliance officer with vast knowledge of SEC Title 17 Chapter II. Your job is to read each section and link other sections, rules, and acts mentioned. What sections, rules, and acts (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) are mentioned in the content? If there are no mentions, return an empty list."
llm.is_function_calling_model = False
extract_prompt = "You are an expert compliance officer with vast knowledge of SEC Title 17 Chapter II. Your job is to find semantic, referential, and literal relationships between the sections. If there are no relationships, return an empty list."
kg_extractor = SimpleLLMPathExtractor(
        extract_prompt=extract_prompt,
        llm=llm,
        max_paths_per_chunk=10,
        num_workers=6,
    )

# random.shuffle(documents)

print("Creating graph store...")
graph_store = create_neo4j_graph_store(neo_url="bolt://localhost:7687", 
                                       password=os.getenv("NEO4J_PWD"), 
                                       config={"connection_timeout": 1000, "connection_acquisition_timeout": 1000, "max_connection_pool_size": 1000})

if not os.path.exists(graph_idx_persist_dir):
    print("Deleting all nodes and relationships...")
    neo4j_query(graph_store, query="""MATCH n=() DETACH DELETE n""")

print("Creating graphrag index...")
graph_index = create_neo4j_graphrag(documents, llm, rules_embed_model, kg_extractor, graph_store, graph_idx_persist_dir=graph_idx_persist_dir, graph_store_persist_dir=graph_store_persist_dir, similarity_top_k=3)

dump_neo4j_database('neo4j', '/workspace/data/')

In [ ]:
query_engine = graph_index.as_query_engine()

In [7]:
neo4j_query(graph_store, query="""MATCH n=() DETACH DELETE n""")
graph_store.close()

In [ ]:
# Vector Database RAG
from llama_index.core.postprocessor import LLMRerank
from rag_utils import create_llama_vector_index_rag

vector_index = create_llama_vector_index_rag(llm, proposal_embed_model, documents=documents_proposal)
query_engine = vector_index.as_query_engine(
    similarity_top_k=3,
    # node_postprocessors=[
    #     LLMRerank(
    #         choice_batch_size=5,
    #         top_n=2,
    #     )
    # ],
    # see https://github.com/run-llama/llama_index/blob/f7c5ee5efbb6172e819f26d1705fcdf6114b11a3/llama-index-core/llama_index/core/response_synthesizers/type.py#L4
    response_mode="tree_summarize", # "accumulate", "compact_accumulate", "compact", "simple_summarize", "tree_summarize"
)

# Settings.chunk_size = 500
# Settings.chunk_overlap = 20

# rules_vector_index = create_llama_vector_index_rag(llm, rules_embed_model, documents=documents)
# graph_index = rules_vector_index.as_query_engine(
#     similarity_top_k=3,
#     # node_postprocessors=[
#     #     LLMRerank(
#     #         choice_batch_size=5,
#     #         top_n=2,
#     #     )
#     # ],
#     # see https://github.com/run-llama/llama_index/blob/f7c5ee5efbb6172e819f26d1705fcdf6114b11a3/llama-index-core/llama_index/core/response_synthesizers/type.py#L4
#     response_mode="tree_summarize", # "accumulate", "compact_accumulate", "compact", "simple_summarize", "tree_summarize"
# )

In [ ]:
response = query_engine.query("Give a summary of the entire proposed rule. What are the key points?")

In [8]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

graph_engine_tools = QueryEngineTool(
            query_engine=graph_index,
            metadata=ToolMetadata(
                name="sec_title_17_chapter_ii_tool",
                description=(
                    "Contains all the current sections, rules, and relationships of SEC Title 17 Chapter II."
                ),
            ),
        )

query_engine_tools = QueryEngineTool(
            query_engine=query_engine,
            metadata=ToolMetadata(
                name="new_rule_proposal_tool",
                description=(
                    "Contains all the information about the newly proposed SEC rule."
                ),
            ),
        )


In [9]:
all_tools = [query_engine_tools, graph_engine_tools]

In [12]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event

#### Define the workflow and all events
class InputEvent(Event):
    input: list[ChatMessage]

class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]

class FunctionOutputEvent(Event):
    output: ToolOutput

In [13]:
from typing import Any, List

from llama_index.core.llms.function_calling import FunctionCallingLLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step


class FuncationCallingAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: FunctionCallingLLM | None = None,
        tools: List[BaseTool] | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm
        assert self.llm.metadata.is_function_calling_model

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.sources = []

    @step
    async def prepare_chat_history(self, ev: StartEvent) -> InputEvent:
        # clear sources
        self.sources = []

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # get chat history
        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

    @step
    async def handle_llm_input(
        self, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input
        '''
        Takes in the chat history, and uses tools to generate a response.
        '''
        response = await self.llm.achat_with_tools(
            self.tools, chat_history=chat_history
        )
        self.memory.put(response.message)

        tool_calls = self.llm.get_tool_calls_from_response(
            response, error_on_no_tool_call=False
        )

        if not tool_calls:
            return StopEvent(
                result={"response": response, "sources": [*self.sources]}
            )
        else:
            return ToolCallEvent(tool_calls=tool_calls)

    @step
    async def handle_tool_calls(self, ev: ToolCallEvent) -> InputEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.get_name(): tool for tool in self.tools}

        tool_msgs = []

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            additional_kwargs = {
                "tool_call_id": tool_call.tool_id,
                "name": tool.metadata.get_name(),
            }
            if not tool:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Tool {tool_call.tool_name} does not exist",
                        additional_kwargs=additional_kwargs,
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                self.sources.append(tool_output)
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=tool_output.content,
                        additional_kwargs=additional_kwargs,
                    )
                )
            except Exception as e:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Encountered error in tool call: {e}",
                        additional_kwargs=additional_kwargs,
                    )
                )

        for msg in tool_msgs:
            self.memory.put(msg)

        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

llm.is_function_calling_model = True

agent = FuncationCallingAgent(
    llm=llm, tools=all_tools, timeout=120, verbose=True
)

ret = await agent.run(input="What is a summary of the proposed rule, and what SEC rules does it change?")

In [ ]:
print(ret["response"])


In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.utils.workflow import draw_most_recent_execution
from IPython.display import display, HTML
draw_all_possible_flows(FuncationCallingAgent, "first_func_agent.html")

In [ ]:
from IPython.display import IFrame

IFrame(src='/workspace/repos/agentic-ai/first_func_agent.html', width=700, height=600)

In [ ]:
from IPython.display import display, HTML

# Read the HTML file content
with open('/workspace/repos/agentic-ai/first_func_agent.html', 'r') as file:
    html_content = file.read()

# Render the HTML content in a Jupyter cell
display(HTML(html_content))

In [5]:
from typing import Optional
import asyncio
from pydantic_settings import BaseSettings
import signal
from llama_deploy.deploy.deploy import (
                        _get_message_queue_client,
                        _deploy_local_message_queue,
                        _get_shutdown_handler
                    )

from llama_deploy import (
    deploy_core,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
    SimpleOrchestratorConfig,
    ControlPlaneServer,
    SimpleOrchestrator,
    LlamaDeployClient
)


In [ ]:
from llama_deploy import (
    deploy_core,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
)


async def main():
    await deploy_core(
        control_plane_config=ControlPlaneConfig(port=8002),
        message_queue_config=SimpleMessageQueueConfig(port=8003),
    )


# if __name__ == "__main__":
#     import asyncio

#     asyncio.run(main())

In [ ]:
from llama_deploy import (
    deploy_workflow,
    WorkflowServiceConfig,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
)
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step


# create a dummy workflow
class MyWorkflow(Workflow):
    @step()
    async def run_step(self, ev: StartEvent) -> StopEvent:
        # Your workflow logic here
        arg1 = str(ev.get("arg1", ""))
        result = arg1 + "_result"
        return StopEvent(result=result)


async def main():
    await deploy_workflow(
        workflow=MyWorkflow(),
        workflow_config=WorkflowServiceConfig(
            host="127.0.0.1", port=8004, service_name="my_workflow"
        ),
        control_plane_config=ControlPlaneConfig(),
    )


if __name__ == "__main__":
    import asyncio

    asyncio.run(main())

In [ ]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
)
import random
from llama_index.core.workflow import draw_all_possible_flows
from llama_index.utils.workflow import draw_most_recent_execution

In [22]:
from typing import Optional
import asyncio
from pydantic_settings import BaseSettings
import signal
from llama_deploy.deploy.deploy import (
                        _get_message_queue_client,
                        _deploy_local_message_queue,
                        _get_shutdown_handler
                    )

from llama_deploy import (
    deploy_core,
    ControlPlaneConfig,
    SimpleMessageQueueConfig,
    SimpleOrchestratorConfig,
    ControlPlaneServer,
    SimpleOrchestrator,
    LlamaDeployClient
)


async def deploy_core(
    control_plane_config: ControlPlaneConfig,
    message_queue_config: BaseSettings,
    orchestrator_config: Optional[SimpleOrchestratorConfig] = None,
) -> None:
    orchestrator_config = orchestrator_config or SimpleOrchestratorConfig()

    message_queue_client = _get_message_queue_client(message_queue_config)

    control_plane = ControlPlaneServer(
        message_queue_client,
        SimpleOrchestrator(**orchestrator_config.model_dump()),
        **control_plane_config.model_dump(),
    )

    message_queue_task = None
    if isinstance(message_queue_config, SimpleMessageQueueConfig):
        message_queue_task = _deploy_local_message_queue(message_queue_config)

    control_plane_task = asyncio.create_task(control_plane.launch_server())

    # let services spin up
    await asyncio.sleep(1)

    # register the control plane as a consumer
    control_plane_consumer_fn = await control_plane.register_to_message_queue()

    consumer_task = asyncio.create_task(control_plane_consumer_fn())

    # let things sync up
    await asyncio.sleep(1)

    # let things run
    if message_queue_task:
        all_tasks = [control_plane_task, consumer_task, message_queue_task]
    else:
        all_tasks = [control_plane_task, consumer_task]

    shutdown_handler = _get_shutdown_handler(all_tasks)
    loop = asyncio.get_event_loop()
    while loop.is_running():
        await asyncio.sleep(0.1)
        signal.signal(signal.SIGINT, shutdown_handler)

        for task in all_tasks:
            if task.done() and task.exception():  # type: ignore
                raise task.exception()  # type: ignore

In [24]:
import httpx
from llama_index.core.workflow import Workflow
from llama_deploy import (
    WorkflowServiceConfig,
    WorkflowService,
)

from llama_deploy.deploy.deploy import (
                        _get_message_queue_config,
                    )

async def deploy_workflow(
    workflow: Workflow,
    workflow_config: WorkflowServiceConfig,
    control_plane_config: ControlPlaneConfig,
) -> None:
    control_plane_url = control_plane_config.url

    async with httpx.AsyncClient() as client:
        response = await client.get(f"{control_plane_url}/queue_config")
        queue_config_dict = response.json()

    message_queue_config = _get_message_queue_config(queue_config_dict)
    message_queue_client = _get_message_queue_client(message_queue_config)

    service = WorkflowService(
        workflow=workflow,
        message_queue=message_queue_client,
        **workflow_config.model_dump(),
    )

    service_task = asyncio.create_task(service.launch_server())

    # let service spin up
    await asyncio.sleep(1)

    # register to message queue
    consumer_fn = await service.register_to_message_queue()

    # register to control plane
    control_plane_url = (
        f"http://{control_plane_config.host}:{control_plane_config.port}"
    )
    await service.register_to_control_plane(control_plane_url)

    # create consumer task
    consumer_task = asyncio.create_task(consumer_fn())

    # let things sync up
    await asyncio.sleep(1)

    all_tasks = [consumer_task, service_task]

    shutdown_handler = _get_shutdown_handler(all_tasks)
    loop = asyncio.get_event_loop()
    while loop.is_running():
        await asyncio.sleep(0.1)
        signal.signal(signal.SIGINT, shutdown_handler)

        for task in all_tasks:
            if task.done() and task.exception():  # type: ignore
                raise task.exception()  # type: ignore

In [27]:
from llama_agents import (
    AgentService,
    ToolService,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=all_tools,
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in all_tools
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=llm,
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Agent that answers questions based on the newly proposed SEC rule.",
    service_name="rule_proposal_agent",
)

In [ ]:
response = agent1.chat('When are written comments on this notice of joint proposed rulemaking need to be submitted?')
print(response.response)

In [ ]:
# questions
# Name of rule 
# Is it a proposed rule or a final rule
# Issue date and Federal Register date
# What agency(ies) is the Rule coming from
# If a proposed rule, when are public comments due by and where should they be sent (this info is in the Rule document under Dates and Addresses)

## Tree summarizer

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(llm=llm, verbose=True)
# prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Create a section to mention the personnel new to AlphaTrAI.
3. Include other highlights and progress made by AlphaTrAI.
4. Ensure the memo and ensure it is factual, optimistic, and any values mention come directly from the text. 

The transcription is as follows:\n{full_doc}"""

response = await summarizer.aget_response(prompt_summary, [doc.text for doc in documents])

In [ ]:
print(response)

## LLM direct summarization

In [ ]:
prompt_summary = f"""You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it the memo has the following items:
1. Extract all the names of new hires and their position, and/or new advisors mentioned in the transcription.
2. Include other highlights and progress made by AlphaTrAI.
3. Ensure the memo is professional, fluid, factual, and optimistic. 

The transcription is as follows:\n{full_doc}"""

response = llm.complete(prompt_summary, max_tokens=5000)

In [ ]:
print(response.text)

## Agentic Summary

In [ ]:
!pip3 install llama-index-embeddings-huggingface llama-index-vector-stores-neo4jvector llama-index-graph-stores-neo4j
!apt install dialog apt-utils -y (done above)
!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | gpg --dearmor -o /etc/apt/keyrings/neotechnology.gpg
!echo 'deb [signed-by=/etc/apt/keyrings/neotechnology.gpg] https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list
!apt list -a neo4j
!add-apt-repository universe -y
!apt install neo4j=1:5.22.0 -y
!echo "neo4j-enterprise neo4j/question select I ACCEPT" | debconf-set-selections
!echo "neo4j-enterprise neo4j/license note" | debconf-set-selections
!apt install openjdk-17-jre -y
!cd /var/lib/neo4j/plugins/ && wget https://github.com/neo4j/apoc/releases/download/5.22.0/apoc-5.22.0-core.jar

In [ ]:
set_neo4j_password('bewaretheneo')
add_lines_to_conf()


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from llama_index.core.agent import ReActAgent
# from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from rag_utils import create_neo4j_graph_store, create_neo4j_graphrag, neo4j_query, set_neo4j_password, add_lines_to_conf
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import nest_asyncio
nest_asyncio.apply()



In [ ]:

llm.is_function_calling_model = True

embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
print("loading embed model...")
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

Settings.embed_model = embed_model
Settings.chunk_size = 300
Settings.chunk_overlap = 50

In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

entities = Literal["PEOPLE", 
                   "PLACE"
]

relations = Literal[
    "ROLE",
    "COMPANY"
]

validation_schema = {
    "People": ["ROLE"],
    "Place": ["COMPANY"],
}


In [ ]:
Settings.chunk_size = 300
Settings.chunk_overlap = 50

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    strict=True,  # if false, will allow triples outside of the schema
    num_workers=4,
    max_triplets_per_chunk=10,
)

graph_store = create_neo4j_graph_store(neo_url="bolt://localhost:7687", 
                                       password=os.getenv("NEO4J_PWD"), 
                                       config={"connection_timeout": 240, "connection_acquisition_timeout": 240, "max_connection_pool_size": 1000})
neo4j_query(graph_store, query="""MATCH (n) DETACH DELETE n""")


graph_index = create_neo4j_graphrag(documents, llm, embed_model, kg_extractor, graph_store)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = QueryEngineTool(
            query_engine=graph_index,
            metadata=ToolMetadata(
                name="graph_tool",
                description=(
                    "Useful for finding people names and roles, and the company they work for."
                ),
            ),
        ),


In [ ]:
!pip3 install llama-agents

In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker, ReActAgentWorker, ReActAgent, LATSAgentWorker



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],#, adding_tool],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]#, adding_tool]
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=llm,
)

worker2 = LATSAgentWorker.from_tools(
    meta_tools,
    llm=llm,
    num_expansions=2,
    max_rollouts=3,
    verbose=True
)

agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Summarize a transcription as a memo for investors and stakeholders.",
    service_name="summarize_transcription",
)